In [1]:
import os
from dotenv import load_dotenv
from langfuse.callback import CallbackHandler
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate
from langfuse.callback import CallbackHandler
from langchain_openai import ChatOpenAI
import pandas as pd
import sys
sys.path.append('../')
print(sys.path)
import prompts as p
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load the .env file
load_dotenv()

# Access the environment variables
os.environ["LANGFUSE_PUBLIC_KEY"] = os.environ.get("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = os.environ.get("LANGFUSE_SECRET_KEY")
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

handler = CallbackHandler()
handler.auth_check()

DATA_PATH = "../../../Datasets/MAMS-ATSA/Downsampled/test/test_single_row_few_shot_fix.csv"
OUTPUT_PATH = "../../../Datasets/Evaluations/Sentiment Analysis/fix_few_shot_pipeline_3.5-turbo.csv"

MODEL = "gpt-3.5-turbo"
MAX_TOKENS = 15
TEMP = 0



['/Users/timgutberlet/My Drive (t.gutberlet01@gmail.com)/05 - Coding/06 - Bachelor Thesis/Error-Analysis-with-LLMs/Sentiment Analysis/Pipelines/single_term', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/timgutberlet/Library/Python/3.12/lib/python/site-packages', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages', '../']


In [2]:
df = pd.read_csv(DATA_PATH)
prompt_name = "prompt_single_term_fewshot"
prompt = p.prompt_single_term_fewshot


df['polarity_pred'] = None
df['prompt_name'] = None
df['prompt'] = None

print(df.head())


   Unnamed: 0  text_id  term_id  \
0           0        0        0   
1           1        0        1   
2           2        0        2   
3           3        0        3   
4           4        1        4   

                                                text  \
0  Amusing details distinguish desserts, from dul...   
1  Amusing details distinguish desserts, from dul...   
2  Amusing details distinguish desserts, from dul...   
3  Amusing details distinguish desserts, from dul...   
4  The server was so busy the night we visited th...   

                          term  polarity  example1_term_id  \
0                     desserts  positive                 0   
1     dulce de leche ice-cream   neutral                 0   
2  chocolate sauce tic-tac-toe   neutral                 0   
3            poached pineapple   neutral                 0   
4                       server  negative                 0   

                                     example1_prompt  example2_term_id  \
0  In

In [3]:
def execute(df, prompt, prompt_name, start = 0):
    if(MODEL == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name = MODEL, temperature = TEMP, max_tokens=MAX_TOKENS, timeout=10)
    else:
        llm = ChatOpenAI(model_name = MODEL, temperature = TEMP, max_tokens=MAX_TOKENS, timeout=10)
    for i in range(start, len(df)):  
        if(df["polarity_pred"][i] != None and df["polarity_pred"][i] != "error"):
            continue
        print(i)
        chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
        example_prompt_1 = df['example1_prompt'][i]
        example_prompt_2 = df['example2_prompt'][i]
        example_prompt_3 = df['example3_prompt'][i]
        
        input_text = df["text"][i]
        entity = df["term"][i]
        try: 
            result = chain.run(example_1 = example_prompt_1, example_2 = example_prompt_2, example_3 = example_prompt_3,  input_text = input_text, entity = entity, callbacks=[handler])
        except Exception as e:
            print(e)
            df.loc[i, 'polarity_pred'] =  'error'
            df.loc[i, 'prompt_name'] = prompt_name
            df.loc[i, 'prompt'] = chain.prompt.format_prompt(example_1 = example_prompt_1, example_2 = example_prompt_2, example_3 = example_prompt_3, input_text = input_text, entity = entity).text
            continue
        handler.langfuse.flush()
        prompt_text = chain.prompt.format_prompt(example_1 = example_prompt_1, example_2 = example_prompt_2, example_3 = example_prompt_3, input_text = input_text, entity = entity).text
        if (i < 5):
            print(prompt_text)
            print("\n")
            print(result)
            print("\n")
        
        if (i % 50 == 0):
            print(str(i) + " of  " + str(len(df)))

        if ("positive" in result.lower()):
            y_pred = 'positive'
        elif ("negative" in result.lower()):
            y_pred = 'negative'
        elif ("neutral" in result.lower()):
            y_pred = 'neutral'
        else:
            y_pred = 'error'

        df.loc[i, 'polarity_pred'] =  y_pred
        df.loc[i, 'prompt_name'] = prompt_name
        df.loc[i, 'prompt'] = prompt_text
    return df


df = execute(df, prompt, prompt_name)
    



0


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


Example 1:
Input: "The decor is not special at all but their food and amazing prices make up for it."
Term: decor
Output: negative


Example 2:
Input: "when tables opened up, the manager sat another party before us."
Term: tables
Output: neutral


Example 3
Input: "service is good although a bit in your face, we were asked every five mins if food was ok, but better that than being ignored."
Term: service
Output: positive

    
Task:
Input: Amusing details distinguish desserts, from dulce de leche ice-cream profiteroles dotting a chocolate sauce tic-tac-toe board, to coconut custard surrounded by a sea of Malibu-rum gelee and poached pineapple.
Prompt: What is the sentiment in the text towards 'desserts'? Only respond with "positive", "negative" or "neutral" as one word.


Positive


0 of  451
1


KeyboardInterrupt: 

In [9]:
df

,Unnamed: 0,text_id,term_id,text,term,polarity,example1_term_id,example1_prompt,example2_term_id,example2_prompt,example3_term_id,example3_prompt,polarity_pred,prompt_name,prompt
0,0,0,0,"Amusing details distinguish desserts, from dul...",desserts,positive,0,"Input: ""The decor is not special at all but th...",3,"Input: ""when tables opened up, the manager sat...",9,"Input: ""service is good although a bit in your...",positive,prompt_single_term_fewshot,"Example 1:\nInput: ""The decor is not special a..."
1,1,0,1,"Amusing details distinguish desserts, from dul...",dulce de leche ice-cream,neutral,0,"Input: ""The decor is not special at all but th...",3,"Input: ""when tables opened up, the manager sat...",9,"Input: ""service is good although a bit in your...",positive,prompt_single_term_fewshot,"Example 1:\nInput: ""The decor is not special a..."
2,2,0,2,"Amusing details distinguish desserts, from dul...",chocolate sauce tic-tac-toe,neutral,0,"Input: ""The decor is not special at all but th...",3,"Input: ""when tables opened up, the manager sat...",9,"Input: ""service is good although a bit in your...",positive,prompt_single_term_fewshot,"Example 1:\nInput: ""The decor is not special a..."
3,3,0,3,"Amusing details distinguish desserts, from dul...",poached pineapple,neutral,0,"Input: ""The decor is not special at all but th...",3,"Input: ""when tables opened up, the manager sat...",9,"Input: ""service is good although a bit in your...",positive,prompt_single_term_fewshot,"Example 1:\nInput: ""The decor is not special a..."
4,4,1,4,The server was so busy the night we visited th...,server,negative,0,"Input: ""The decor is not special at all but th...",3,"Input: ""when tables opened up, the manager sat...",9,"Input: ""service is good although a bit in your...",negative,prompt_single_term_fewshot,"Example 1:\nInput: ""The decor is not special a..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,446,169,446,The server came by only once to pour additiona...,server,negative,0,"Input: ""The decor is not special at all but th...",3,"Input: ""when tables opened up, the manager sat...",9,"Input: ""service is good although a bit in your...",negative,prompt_single_term_fewshot,"Example 1:\nInput: ""The decor is not special a..."
447,447,169,447,The server came by only once to pour additiona...,wine,neutral,0,"Input: ""The decor is not special at all but th...",3,"Input: ""when tables opened up, the manager sat...",9,"Input: ""service is good although a bit in your...",negative,prompt_single_term_fewshot,"Example 1:\nInput: ""The decor is not special a..."
448,448,169,448,The server came by only once to pour additiona...,the table,neutral,0,"Input: ""The decor is not special at all but th...",3,"Input: ""when tables opened up, the manager sat...",9,"Input: ""service is good although a bit in your...",negative,prompt_single_term_fewshot,"Example 1:\nInput: ""The decor is not special a..."
449,449,169,449,The server came by only once to pour additiona...,fish,neutral,0,"Input: ""The decor is not special at all but th...",3,"Input: ""when tables opened up, the manager sat...",9,"Input: ""service is good although a bit in your...",negative,prompt_single_term_fewshot,"Example 1:\nInput: ""The decor is not special a..."


In [5]:
df.to_csv(OUTPUT_PATH, index = True)